[View in Colaboratory](https://colab.research.google.com/github/gweiying/SFD_pytorch/blob/master/Copy_of_Finetuning_edit.ipynb)

In [0]:
from google.colab import files

uploaded = files.upload() 

Saving net_s3fd_gender_cuda.py to net_s3fd_gender_cuda (1).py


In [0]:
!git clone https://github.com/gweiying/SFD_pytorch.git 
!pip3 install torch
!pip3 install torchvision

import zipfile
zip_ref = zipfile.ZipFile('./SFD_pytorch/s3fd_convert.pth.zip', 'r')
zip_ref.extractall('./SFD_pytorch/')
zip_ref.close()

fatal: destination path 'SFD_pytorch' already exists and is not an empty directory.


In [0]:
from __future__ import print_function

import torch
import torch.optim as optim

from torch.utils.data.dataset import Dataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.autograd import Variable
torch.backends.cudnn.bencmark = True
import torchvision
import torchvision.transforms as transforms

import os,sys,cv2,random,datetime,time,math
import argparse
import numpy as np

from net_s3fd_gender_cuda import *
from SFD_pytorch.bbox import *
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image

In [0]:

class CelebDataset(Dataset):
    """Dataset wrapping images and target labels
    Arguments:
        A CSV file path
        Path to image folder
        Extension of images
        PIL transforms
    """

    def __init__(self, csv_path, img_path, img_ext, transform=None):
    
        tmp_df = pd.read_csv(csv_path)
        assert tmp_df['Image_Name'].apply(lambda x: os.path.isfile(img_path + x + img_ext)).all(), \
"Some images referenced in the CSV file were not found"
        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df['Image_Name']
        self.y_train = self.mlb.fit_transform(tmp_df['Gender'].str.split()).astype(np.float32)
    
    def __getitem__(self, index):
        img = cv2.imread(self.img_path + self.X_train[index] + self.img_ext)
        img = cv2.resize(img, (256,256))
        img = img - np.array([104,117,123])
        img = img.transpose(2, 0, 1)
        
        #img = img.reshape((1,)+img.shape)
        img = torch.from_numpy(img).float()
        #img = Variable(torch.from_numpy(img).float(),volatile=True)
        
        #if self.transform is not None:
        #    img = self.transform(img)
        
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        return len(self.X_train.index)

In [0]:
train_data = "./SFD_pytorch/index.csv"
img_path = "./SFD_pytorch/data/Celeb_Small_Dataset/"
img_ext = ".jpg"
dset = CelebDataset(train_data,img_path,img_ext,None)
train_loader = DataLoader(dset,
                          batch_size=1,
                          shuffle=True,
                          num_workers=1 # 1 for CUDA
                         # pin_memory = True # CUDA only
                         )

In [0]:
def save(model, optimizer, loss, filename):
    save_dict = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss.data[0]
        }
    torch.save(save_dict, filename)

In [0]:
def train_model(model, criterion, optimizer, num_classes, num_epochs = 100):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        model.train()
        running_loss = 0.0

        for i,(img,label) in enumerate(train_loader):
            img = img.view((1,)+img.shape[1:])
            if use_cuda:
                data, target = Variable(img.cuda()), Variable(torch.Tensor(label).cuda())
            else:
                data, target = Variable(img), Variable(torch.Tensor(label))
            target = target.view(1,num_classes)

            optimizer.zero_grad()
            olist = model(data)
            for p in range(4): olist[p*2+1] = F.softmax(olist[p*2+1], dim = 1)
            genscorelist = []
            loss = 0
            for p in range(4):
                ocls = olist[p*2].data.cuda()
                FB,FC,FH,FW = ocls.size()
                stride = 2**(p+3) #8,16,32,64 ??
                anchor = stride*4
                for Findex in range(FH*FW):
                    windex,hindex = Findex%FW,Findex//FW
                    axc,ayc = stride/2+windex*stride,stride/2+hindex*stride
                    score = ocls[0,1,hindex,windex]
                    if score<0.05:continue
                    #if use_cuda:
                    #olist[p*2+1] = Variable(torch.Tensor(olist[p*2+1]),requires_grad=True)
                    #else:
                    #  olist[p*2+1] = Variable(torch.Tensor((olist[p*2+1])),requires_grad=True)
                    loss+= criterion(olist[p*2+1],target)
            
            loss.backward()
            optimizer.step()
            running_loss += loss.data[0]

            
            #loss = 0
            #for p in genscorelist:
            #    if use_cuda:
            #      p = Variable(torch.Tensor(p.cuda()),requires_grad=True)
            #    else:
            #      p = Variable(torch.Tensor(p),requires_grad=True)
            #    loss += criterion(p,target)
            
            
            
            if i%50==0:
                print("Reached iteration ",i)
                #running_loss += loss.data[0]
                
            
            
            
        if epoch % 10 == 0:
            save(model, optimizer, loss, 'faceRecog.saved.model')
        print(running_loss)


In [0]:
num_classes = 2
myModel = s3fd(num_classes)
loadedModel = torch.load('./SFD_pytorch/s3fd_convert.pth')
newModel = myModel.state_dict()
pretrained_dict = {k: v for k, v in loadedModel.items() if k in newModel}
newModel.update(pretrained_dict)
myModel.load_state_dict(newModel)

In [8]:
use_cuda = True
myModel.eval()

s3fd(
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_3): Conv2d(512, 512

In [0]:
criterion = nn.MSELoss()

for param in myModel.parameters():
    param.requires_grad = False
    
#extract weights to a new tensor and add dimension
hold_conv4_3 = myModel.conv4_3_norm_mbox_conf.weight[0].data.unsqueeze(0)
hold_conv5_3 = myModel.conv5_3_norm_mbox_conf.weight[0].data.unsqueeze(0)
hold_fc7 = myModel.fc7_mbox_conf.weight[0].data.unsqueeze(0)
hold_conv6_2 = myModel.conv6_2_mbox_conf.weight[0].data.unsqueeze(0)

#concatenate tensor and add to layer's weight
myModel.conv4_3_norm_gender.weight = torch.nn.Parameter(torch.cat((hold_conv4_3, hold_conv4_3),0))
myModel.conv5_3_norm_gender.weight = torch.nn.Parameter(torch.cat((hold_conv5_3, hold_conv5_3),0))
myModel.fc7_norm_gender.weight = torch.nn.Parameter(torch.cat((hold_fc7, hold_fc7), 0))
myModel.conv6_2_norm_gender.weight = torch.nn.Parameter(torch.cat((hold_conv6_2, hold_conv6_2), 0))

for q in myModel.conv4_3_norm_gender.parameters():
    q.requires_grad = True
    
for q in myModel.conv5_3_norm_gender.parameters():
    q.requires_grad = True
    
for q in myModel.fc7_norm_gender.parameters():
    q.requires_grad = True
    
for q in myModel.conv6_2_norm_gender.parameters():
    q.requires_grad = True
    
for q in myModel.fc4_3.parameters():
    q.requires_grad = True

for q in myModel.fc5_3.parameters():
    q.requires_grad = True
    
for q in myModel.ffc7.parameters():
    q.requires_grad = True
    
for q in myModel.fc6_2.parameters():
    q.requires_grad = True



In [10]:
print(myModel.conv4_3_norm_gender.weight.size())
print(myModel.conv5_3_norm_gender.weight.size())
print(myModel.fc7_norm_gender.weight.size())
print(myModel.conv6_2_norm_gender.weight.size())

torch.Size([2, 512, 3, 3])
torch.Size([2, 512, 3, 3])
torch.Size([2, 1024, 3, 3])
torch.Size([2, 512, 3, 3])


Comment: I ran this for over 200 epoches to get the results here I just didn't display them!!


In [0]:
optimizer = optim.SGD(filter(lambda p: p.requires_grad,myModel.parameters()), lr=0.00001, momentum=0.9)

myModel = myModel.cuda()
model_ft = train_model(myModel, criterion, optimizer, num_classes, num_epochs=301)

Epoch 0/300
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  """


tensor(527.9679, device='cuda:0')
Epoch 1/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(462.4369, device='cuda:0')
Epoch 2/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(466.1803, device='cuda:0')
Epoch 3/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(430.3749, device='cuda:0')
Epoch 4/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(424.6318, device='cuda:0')
Epoch 5/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(421.6869, device='cuda:0')
Epoch 6/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(414.9211, device='cuda:0')
Epoch 7/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
R

Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(371.9951, device='cuda:0')
Epoch 18/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(369.5359, device='cuda:0')
Epoch 19/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(377.3501, device='cuda:0')
Epoch 20/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(364.2359, device='cuda:0')
Epoch 21/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(362.7145, device='cuda:0')
Epoch 22/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(377.0851, device='cuda:0')
Epoch 23/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(362.5345, device='cuda

Reached iteration  100
Reached iteration  150
tensor(340.5747, device='cuda:0')
Epoch 34/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(338.4427, device='cuda:0')
Epoch 35/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(342.1156, device='cuda:0')
Epoch 36/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(334.0699, device='cuda:0')
Epoch 37/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(328.1235, device='cuda:0')
Epoch 38/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(332.7679, device='cuda:0')
Epoch 39/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(337.6722, device='cuda:0')
Epoch 40/300
----------
Reached iterat

Reached iteration  150
tensor(312.6721, device='cuda:0')
Epoch 50/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(312.9674, device='cuda:0')
Epoch 51/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(307.5961, device='cuda:0')
Epoch 52/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(309.3677, device='cuda:0')
Epoch 53/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(301.5646, device='cuda:0')
Epoch 54/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(304.3327, device='cuda:0')
Epoch 55/300
----------
Reached iteration  0
Reached iteration  50
Reached iteration  100
Reached iteration  150
tensor(304.8016, device='cuda:0')
Epoch 56/300
----------
Reached iteration  0
Reached iteratio

In [0]:
def transform(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (256,256))
        img = img - np.array([104,117,123])
        img = img.transpose(2, 0, 1)
        
        img = img.reshape((1,)+img.shape)
        img = torch.from_numpy(img).float()
        
        return Variable(img.cuda())

testImage1 = transform('./SFD_pytorch/data/Test/TestCeleb_4/26-FaceId-0.jpg')#F
testImage2 = transform('./SFD_pytorch/data/Test/TestCeleb_6/23-FaceId-0.jpg')#M
testImage3 = transform('./SFD_pytorch/data/Test/TestCeleb_1/23-FaceId-0.jpg')#F
testImage4 = transform('./SFD_pytorch/data/Test/TestCeleb_8/24-FaceId-0.jpg')#M
testImage5 = transform('./SFD_pytorch/data/Test/TestCeleb-2/27-FaceId-0.jpg')#M
testImage6 = transform('./SFD_pytorch/data/Test/TestCeleb_5/23-FaceId-0.jpg')#F


testImageList = [testImage1,testImage2,testImage3,testImage4,testImage5,testImage6]
ctr = 1
for testImage in testImageList:
  print("Test image" + str(ctr))
  olist = myModel(testImage)
  for p in range(int(len(olist)/2)): olist[p*2+1] = F.softmax(olist[p*2+1], dim = 1)
  for p in range(int(len(olist)/2)):
      ocls,ogen = olist[p*2].data.cpu(),olist[p*2+1].data.cpu()
      FB, FC, FH, FW= ocls.size()
      stride = 2**(p+3) #8,16,32,64 ??
      anchor = stride*4
      for Findex in range(FH*FW):
          windex,hindex = Findex%FW,Findex//FW
          axc,ayc = stride/2+windex*stride,stride/2+hindex*stride
          score = ocls[0,1,hindex,windex]
          if score<0.05: continue
          print(ogen)
          break    
  ctr+=1

#testImage2 = transform('./genderrecognition/data/Test/TestCeleb_4/26-FaceId-0.jpg')
#testImage3 = transform('./genderrecognition/data/Test/TestCeleb_4/27-FaceId-0.jpg')
#testImage4 = transform('./genderrecognition/data/Test/TestCeleb_10/25-FaceId-0.jpg')
#testImage5 = transform('./genderrecognition/data/Test/TestCeleb_10/26-FaceId-0.jpg')
#testImage6 = transform('./genderrecognition/data/Test/TestCeleb_10/24-FaceId-0.jpg')   
        
# output2 = myModel(testImage2)
# output3 = myModel(testImage2)
# output4 = myModel(testImage4)
# output5 = myModel(testImage5)
# output6 = myModel(testImage6)
# print("testImage1 - ",output1)
# print("testImage2 - ",output2)
# print("testImage3 - ",output3)
# print("testImage1 - ",output4)
# print("testImage2 - ",output5)
# print("testImage3 - ",output6)"""

Test image1
tensor([[ 0.7344,  0.2656]])
Test image2
tensor([[ 0.0369,  0.9631]])
tensor([[ 0.2092,  0.7908]])
Test image3
tensor([[ 0.1944,  0.8056]])
tensor([[ 0.2513,  0.7487]])
Test image4
tensor([[ 0.0913,  0.9087]])
Test image5
tensor([[ 0.3118,  0.6882]])
Test image6
tensor([[ 0.7891,  0.2109]])
tensor([[ 0.3515,  0.6485]])


In [0]:
criterion = nn.MSELoss().cuda()

num_classes = 2
myModel = s3fd(num_classes)
file = torch.load('faceRecog.saved.model')
pretrained_dict = file['model_state_dict']




for param in myModel.parameters():
    param.requires_grad = False
    

for q in myModel.conv4_3_norm_gender.parameters():
    q.requires_grad = True
    
for q in myModel.conv5_3_norm_gender.parameters():
    q.requires_grad = True
    
for q in myModel.fc7_norm_gender.parameters():
    q.requires_grad = True
    
for q in myModel.conv6_2_norm_gender.parameters():
    q.requires_grad = True
    
for q in myModel.fc4_3.parameters():
    q.requires_grad = True

for q in myModel.fc5_3.parameters():
    q.requires_grad = True
    
for q in myModel.ffc7.parameters():
    q.requires_grad = True
    
for q in myModel.fc6_2.parameters():
    q.requires_grad = True
    
myModel.load_state_dict(pretrained_dict)


use_cuda = True
myModel.eval()

In [0]:
optimizer = optim.SGD(filter(lambda p: p.requires_grad,myModel.parameters()), lr=0.001, momentum=0.9)
if use_cuda:
    myModel = myModel.cuda()
model_ft = train_model(myModel, criterion, optimizer, num_classes, num_epochs=20 )

In [0]:
ls